# STAR PERFORMERS

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from bs4 import BeautifulSoup
import xmltodict
from pprint import pprint
import os
import csv
import datetime
from api_keys import z_keys

In [ ]:
ny18_df = pd.read_csv("Files/2018NYSales.csv")

In [ ]:
len(ny18_df)

In [ ]:
ny18_df = ny18_df.rename(columns={' SALE PRICE ': 'SALE PRICE'})

In [ ]:
ny18_df.columns

In [ ]:
ny18_df.describe()

In [ ]:
ny18_df["ADDRESS"] = ny18_df["ADDRESS"].str.strip()
ny18_df['ADDRESS'] = ny18_df['ADDRESS'].str.split(' ').apply(lambda x: '+'.join(x))
ny18_df.count()

In [ ]:
ny18_df["SALE PRICE"] = ny18_df.loc[:,"SALE PRICE"].replace('[\$,]', '', regex=True)
ny18_df["SALE PRICE"] = ny18_df.loc[:,"SALE PRICE"].replace('[-,]', '0', regex=True)

In [ ]:
# Removing home prices less than 100K to avoid a bias after preliminary research on what these lower sale prices rep.
# like: Internal family sale transfer at $0, garage sale etc.

ny18_df = ny18_df[ny18_df['SALE PRICE'].astype(float) > 100000]
ny18_df = ny18_df.drop_duplicates(subset=['ADDRESS'], keep = 'first')

In [ ]:
len(ny18_df)

In [ ]:
ny18_df.head()

In [ ]:
# response = requests.get(f"https://www.zillow.com/webservice/GetDeepSearchResults.htm?zws-id={z_keys}&address=635+west+42+street&citystatezip=10036&rentzestimate=true")

In [ ]:
# print(response)

In [ ]:
#Using BeautifulSoup to convert XML text to pretty format
# soup = BeautifulSoup(response.text)
print(soup.prettify())

In [ ]:
doc = xmltodict.parse(response.text)['SearchResults:searchresults']['response']['results']['result']
# len(doc)

In [ ]:
def check_data(address):
    return (address.get('rentzestimate')) and (address.get('lastSoldPrice'))

In [ ]:
def need_rent_estimate(address):
    return (address.get('rentzestimate')) and not(address.get('lastSoldPrice'))

In [ ]:
def need_sale_estimate(address):
    return (not(address.get('rentzestimate')) and (address.get('lastSoldPrice')))

In [ ]:
def get_estimate(address, zipcode):
    print("Entering get_estimate")
    rent = address['rentzestimate']['amount']['#text']
    sold = address['lastSoldPrice']['#text']
    sold_year = datetime.datetime.strptime(address['lastSoldDate'], '%m/%d/%Y').date().year
#     zestimate=address['zestimate']['amount']['#text']
    sqft = address['finishedSqFt']
    address = address['address']['street']
    var = 0
    return pd.Series({'address':address, 'zipcode':zipcode, 'sold':sold, 'rent':rent, 'sold year':sold_year, 'sqft':sqft, 'var':var})     

In [ ]:
def get_rent_estimate(address, zipcode):
    print("Entering get_rent_estimate")
    rent = address['rentzestimate']['amount']['#text']
    address = address['address']['street']
    var = 2
    return pd.Series({'address':address, 'zipcode':zipcode, 'rent':rent, 'var':var})

In [ ]:
def get_property_estimate(address, zipcode):
    print("Entering get_property_estimate")
    sold = address['lastSoldPrice']['#text']
    sold_year = datetime.datetime.strptime(address['lastSoldDate'], '%m/%d/%Y').date().year
#     zestimate=address['zestimate']['amount']['#text']
    sqft = address['finishedSqFt']
    address = address['address']['street']
    var = 1
    return pd.Series({'address':address, 'zipcode':zipcode, 'sold':sold, 'sold year':sold_year, 'sqft':sqft, 'var':var}) 

In [ ]:
ny18_df = ny18_df.reset_index()
ny18_df.head()

In [ ]:
ny18_df.loc[0,'BLOCK']

In [ ]:
block =[]
lot=[]
ny_sale_amt =[]
ny_sale_date=[]
series = []
curdoc={}
counter = 0

for address, zipcode in zip(ny18_df['ADDRESS'],ny18_df['ZIP CODE']):
    counter +=1
    try:
       i = 0
       print(f"https://www.zillow.com/webservice/GetDeepSearchResults.htm?zws-id={z_keys}&address={address}&citystatezip={zipcode}&rentzestimate=true counter={counter}")
       response_main = requests.get(f"https://www.zillow.com/webservice/GetDeepSearchResults.htm?zws-id={z_keys}&address={address}&citystatezip={zipcode}&rentzestimate=true")
#        block=ny18_Update['BLOCK']
#        lot=ny18_Update['LOT']
#        sale_amt=ny18_Update['SALE PRICE']
#        sale_date=ny18_Update['SALE DATE']
       doc = xmltodict.parse(response_main.text, force_list={'result': True})['SearchResults:searchresults']['response']['results']['result']

       while i < len(doc):
           print(f"Record {i}: {doc[i]['address']['street']}")
           
           if check_data(doc[i]):
               new_series = get_estimate(doc[i], zipcode)
               series.append(new_series)
           elif need_rent_estimate(doc[i]):
               new_series = get_rent_estimate(doc[i], zipcode)
               series.append(new_series)
           elif need_sale_estimate(doc[i]):
               new_series = get_property_estimate(doc[i], zipcode)
               series.append(new_series)
           else:
               print("NO ESTIMATED FUNCTIONS ENTERED!!")
                 
           i+=1
                 
           print(f'End of record {i}')
           print('-------------------------------\n')
       time.sleep(1)
       df = pd.concat(series, 1,sort=True).T
    except Exception as e: print(e)

In [ ]:
# xmltodict.parse(response_main.text)['SearchResults:searchresults'].keys()

In [ ]:
df.count()

In [ ]:
df.to_csv("OutputData.csv")

In [ ]:
# response1 = requests.get(f"https://www.zillow.com/webservice/GetDeepSearchResults.htm?{z_keys}&address=520+WEST+23RD+STREET,+12D&citystatezip=10011&rentzestimate=true")
# doc1 = xmltodict.parse(response1.text, force_list={'result': True})['SearchResults:searchresults']['response']['results']['result']

# d2 = datetime.datetime.strptime(doc1[0]['lastSoldDate'], '%m/%d/%Y').date()

# d1 = date(2018, 1, 1)
# d3 = date(2018, 12, 31)

# if d1 < d2 < d3:
#     print(f"True {d3} > {d2} > {d1}")
# else:
#     print("False")